In [1]:
# Library or Package required
import pandas as pd


def explain(a):
    print(a.__doc__)

# Data Preparation
## Data Extraction and Data Integration

In [2]:
# To extract Malaysia export data from year 2000 to year 2022
exp00_04 = pd.read_excel("./Dataset/2000-2004.xlsx", header=5)
exp05_09 = pd.read_excel("./Dataset/2005-2009.xlsx", header=5)
exp10_14 = pd.read_excel("./Dataset/2010-2014.xlsx", header=5)
exp15_19 = pd.read_excel("./Dataset/2015-2019.xlsx", header=5)
exp20_22 = pd.read_excel("./Dataset/2020-2022.xlsx", header=5)

export = [exp00_04, exp05_09, exp10_14, exp15_19, exp20_22]
year_list = []
export_list = []

# To integrate the extracted dataset into one unified dataset
for item in export:
    # To transpose the column and rows
    melted_item = pd.melt(item.tail(1), var_name = 'year', value_name = 'export')
    
    # To convert data into numeric and remove non-numeric data
    melted_item = melted_item.apply(pd.to_numeric, errors = 'coerce')
    melted_item = melted_item.dropna()
    
    # To convert the year data into date format
    melted_item['year'] = pd.to_datetime(melted_item['year'], format= '%Y').dt.year
    
    # To reset the dataframe index
    clean_data1 = melted_item.reset_index(drop = True)
    
    # To convert the column data into list
    year_list.extend(clean_data1['year'].to_list())
    export_list.extend(clean_data1['export'].to_list())
    
# Create a dictionary from the lists
data1 = {'year' : year_list,
        'export' : export_list}

# Create a dataframe from the unified dataset
# To create dataframe based on the merged dataset
df1 = pd.DataFrame(data1)
display(df1.head())

,year,export
0,2000,3.732703e+11
1,2001,3.342838e+11
2,2002,3.574300e+11
3,2003,3.978844e+11
4,2004,4.812530e+11


In [3]:
# Extract other datasets (exchange rate, real effective exchange rate, population, gdp)
ex = pd.read_csv("./Dataset/Currency.csv", header=2)
reer = pd.read_excel("./Dataset/Real Effective Exchange Rate.xlsx", header=3)
pop = pd.read_csv("./Dataset/Population.csv", header=2)
gdp = pd.read_csv("./Dataset/GDP.csv", header=2)

# To filter out Malaysia's data
ex = ex[ex['Country Name'] == 'Malaysia']
# To transpose the column and rows
melted_ex = pd.melt(ex, var_name = 'year',  value_name = 'ER')

# To filter out Malaysia's data
reer = reer[reer['Country Name'] == 'Malaysia']
# To transpose the column and rows
melted_reer = pd.melt(reer, var_name = 'year', value_name = 'REER')

# To filter out Malaysia's data
pop = pop[pop['Country Name'] == 'Malaysia']
# To transpose the column and rows
melted_pop = pd.melt(pop, var_name = 'year', value_name = 'Pop')

# To filter out Malaysia's data
gdp = gdp[gdp['Country Name'] == 'Malaysia']
# To transpose the column and rows
melted_gdp = pd.melt(gdp, var_name = 'year', value_name = 'GDP')

variable = [melted_ex, melted_reer, melted_pop, melted_gdp]
data2 = {}

# To merge the extracted variables into one unified dataset
for item in variable:
    # To convert data into numeric and remove non-numeric data
    item = item.apply(pd.to_numeric, errors= 'coerce').dropna()
    
    # To convert the year data into date format    
    item['year'] = pd.to_datetime(item['year'], format= '%Y').dt.year
    
    # To filter out data for year 2000 onwards
    item = item[item['year'] >= 2000]
    
    # To reset the dataframe index    
    clean_data2 = item.reset_index(drop = True)
    result = clean_data2.to_dict(orient='list')
    data2.update(result)
    
# To create dataframe based on the merged dataset
df2 = pd.DataFrame(data2)
display(df2.head())

,year,ER,REER,Pop,GDP
0,2000,3.8,98.502240,22945150.0,9.378974e+10
1,2001,3.8,103.338160,23542517.0,9.278395e+10
2,2002,3.8,103.469925,24142445.0,1.008455e+11
3,2003,3.8,97.881675,24739411.0,1.102024e+11
4,2004,3.8,93.476392,25333247.0,1.247495e+11


In [4]:
# To merge df1 and df2
final_df = pd.merge(df1, df2)

# New column for MY export in USD currency and remove the initial export value
final_df['Export'] = final_df['export']/final_df['ER']
final_df = final_df.drop('export', axis = 1)

# To assign units to the dataset
final_df['Pop'] = final_df['Pop']/10**6 # in millions
final_df['GDP'] = final_df['GDP']/10**9 # in billions
final_df['Export'] = final_df['Export']/10**9 # in billions

display(final_df.head())
final_df.info()

,year,ER,REER,Pop,GDP,Export
0,2000,3.8,98.502240,22.945150,93.789737,98.229030
1,2001,3.8,103.338160,23.542517,92.783947,87.969425
2,2002,3.8,103.469925,24.142445,100.845526,94.060531
3,2003,3.8,97.881675,24.739411,110.202368,104.706419
4,2004,3.8,93.476392,25.333247,124.749474,126.645524


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23 entries, 0 to 22
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   year    23 non-null     int64  
 1   ER      23 non-null     float64
 2   REER    23 non-null     float64
 3   Pop     23 non-null     float64
 4   GDP     23 non-null     float64
 5   Export  23 non-null     float64
dtypes: float64(5), int64(1)
memory usage: 1.2 KB
